In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../")

import pykep
import numpy as np
import matplotlib.pyplot as plt

starting_t = pykep.epoch_from_string('2022-01-01 00:00:00.000')
lower_cutoff_in_km = 6371 + 200 # Earth radius + ...
higher_cutoff_in_km = 6371 + 2000

## Read TLE data

In [ ]:
planet_list = pykep.util.read_tle("../data/tle.txt")
#satcat = pykep.util.read_satcat("../data/satcat.txt")
print("Loaded ",len(planet_list)," planets.")

## Plot some examples

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d');
for i in range (10):
    pykep.orbit_plots.plot_planet(planet_list[i],axes=ax)

## Propagate all objects to t and discard too low and high ones

In [ ]:
objects = []
count_too_low = 0
count_too_high = 0
for planet in planet_list:
    try:
        pos,v = planet.eph(starting_t)
        
        # convert to km and numpy
        pos = np.asarray(pos) / 1000.0 
        v = np.asarray(v) / 1000.0
        altitude = np.linalg.norm(pos)
        if altitude < lower_cutoff_in_km:
            count_too_low += 1
            continue
        if altitude > higher_cutoff_in_km:
            count_too_high += 1
            continue
        
        objects.append((pos,v))
    except RuntimeError as e:
        print(e, " propagating ",planet.name)
        
print("Successfully propagated ",len(objects)," objects.")
print(count_too_low," had a too small altitude")
print(count_too_high," had a too high altitude")

## Plot and store results

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d');

positions = np.array([pos for pos,_ in objects])
velocities = np.array([v for _,v in objects])

ax.scatter(positions[:,0],positions[:,1],positions[:,2],".",alpha=0.25)

In [ ]:
np.savetxt("../data/pos.csv",positions)
np.savetxt("../data/v.csv",velocities)